## Searching for bouts for a day of ephys recording
- mircophone chan and sync data and bouts are extracted in sglx_preprocess-s_b1253_21-multisess-pouli-ephys

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-12-03 17:35:33,971 root         INFO     Running on pouli


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs

### Get the file locations for a session (day) of recordings

In [3]:
reload(et)
sess_par = {'bird': 's_b1253_21',
           'sess': '2021-06-13',
           'sort': 2}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='sglx')

raw_folder = exp_struct['folders']['sglx']

derived_folder = exp_struct['folders']['derived']

bouts_folder = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx')

sess_bouts_file = os.path.join(bouts_folder, 'bout_sess_auto.pickle')
sess_bouts_curated_file = os.path.join(bouts_folder, 'bout_curated.pickle')

#os.makedirs(bouts_folder, exist_ok=True)

In [4]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1253_21',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-06-13',
 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-06-13/sglx',
 'kwik': '/experiment/s_b1253_21/sglx/kwik/2021-06-13',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1253_21/2021-06-13/sglx',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx',
 'tmp': '/experiment/tmp/tmp',
 'msort': '/experiment/tmp/s_b1253_21/sglx/msort/2021-06-13',
 'ksort': '/experiment/tmp/s_b1253_21/sglx/ksort/2021-06-13/',
 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/'}

### load concatenated the files of the session

In [5]:
def read_session_auto_bouts(exp_struct):
    # list all files of the session
    # read into list of pandas dataframes and concatenate
    # read the search parameters of the first session
    # return the big pd and the search params
    derived_folder = exp_struct['folders']['derived']
    
    search_params_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_search_params.pickle')
    hparams=None
    with open(search_params_files[0], 'rb') as fh:
        hparams = pickle.load(fh)
    
    sess_bout_pd_file = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx', 'bout_auto.pickle')
    try:
        bout_pd = pd.read_pickle(sess_bout_pd_file)
    except FileNotFoundError:
        logger.info('Not found session pandas file {}, loading each epoch file and concatenating'.format(sess_bout_pd_file))
        bout_pd_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_auto.pickle')
        logger.info('Files to load {}'.format(bout_pd_files))
        bout_pd = pd.concat([pd.read_pickle(p) for p in bout_pd_files[:]])
    
    bout_pd.reset_index(inplace=True, drop=True)
    return bout_pd, hparams

bout_pd, hparams = read_session_auto_bouts(exp_struct)

In [6]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      414 non-null    int64  
 1   end_ms        414 non-null    int64  
 2   start_sample  414 non-null    int64  
 3   end_sample    414 non-null    int64  
 4   p_step        414 non-null    object 
 5   rms_p         414 non-null    float64
 6   peak_p        414 non-null    float64
 7   bout_check    414 non-null    bool   
 8   file          414 non-null    object 
 9   len_ms        414 non-null    int64  
 10  syl_in        414 non-null    object 
 11  n_syl         414 non-null    int64  
 12  peaks_p       414 non-null    object 
 13  n_peaks       414 non-null    int64  
 14  l_p_ratio     414 non-null    float64
 15  waveform      414 non-null    object 
 16  confusing     414 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 49.4+ KB


In [7]:
np.unique(bout_pd['start_ms']).shape

(414,)

In [8]:
### list all the source files
sess_wav_files = np.unique(bout_pd['file'].values)
sess_wav_files

array(['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/0709_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/1227_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/1819_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/2342_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/713_g0/wav_mic.wav'],
      dtype=object)

#### optional: drop the night recordings

In [9]:
night_file = sess_wav_files[-2]
bout_pd.drop(bout_pd[bout_pd['file']==night_file].index, inplace=True)
print(night_file)

/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/2342_g0/wav_mic.wav


In [10]:
np.unique(bout_pd['file'].values)

array(['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/0709_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/1227_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/1819_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/713_g0/wav_mic.wav'],
      dtype=object)

In [11]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 413
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      193 non-null    int64  
 1   end_ms        193 non-null    int64  
 2   start_sample  193 non-null    int64  
 3   end_sample    193 non-null    int64  
 4   p_step        193 non-null    object 
 5   rms_p         193 non-null    float64
 6   peak_p        193 non-null    float64
 7   bout_check    193 non-null    bool   
 8   file          193 non-null    object 
 9   len_ms        193 non-null    int64  
 10  syl_in        193 non-null    object 
 11  n_syl         193 non-null    int64  
 12  peaks_p       193 non-null    object 
 13  n_peaks       193 non-null    int64  
 14  l_p_ratio     193 non-null    float64
 15  waveform      193 non-null    object 
 16  confusing     193 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 24.5+ KB


In [12]:
np.unique(bout_pd['start_ms']).size

193

###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [13]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [14]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    ## fill in the epoch
    bout_pd['epoch'] = bout_pd['file'].apply(lambda x: et.split_path(x)[-2])
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)
    
    # set all to 'confusing' (unchecked) and 'bout_check' false (not a bout)
    bout_pd['confusing'] = True
    bout_pd['bout_check'] = False

cleanup(bout_pd)

In [15]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_ms        192 non-null    int64  
 1   end_ms          192 non-null    int64  
 2   start_sample    192 non-null    int64  
 3   end_sample      192 non-null    int64  
 4   p_step          192 non-null    object 
 5   rms_p           192 non-null    float64
 6   peak_p          192 non-null    float64
 7   bout_check      192 non-null    bool   
 8   file            192 non-null    object 
 9   len_ms          192 non-null    int64  
 10  syl_in          192 non-null    object 
 11  n_syl           192 non-null    int64  
 12  peaks_p         192 non-null    object 
 13  n_peaks         192 non-null    int64  
 14  l_p_ratio       192 non-null    float64
 15  waveform        192 non-null    object 
 16  confusing       192 non-null    bool   
 17  valid_waveform  192 non-null    boo

In [16]:
np.unique(bout_pd['start_ms']).shape

(192,)

In [17]:
bout_pd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch
0,21460,32220,429200,644400,"[8.713733042828666, 130.93567831255595, 233.87...",21.222267,359.953578,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,10760,"[[21460, 21495], [22305, 22340], [24145, 24190...",17,"[2, 30, 105, 128, 172, 180, 198, 211, 540, 609...",58,185.517241,"[1065, 1076, 1081, 1095, 1083, 1103, 1086, 108...",True,True,True,0709_g0
1,40125,54645,802500,1092900,"[13.096523956130605, 9.062461794160846, 31.203...",21.222267,310.517498,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,14520,"[[40125, 40155], [41055, 41095], [42120, 42165...",18,"[4, 11, 18, 178, 187, 200, 400, 414, 432, 465,...",60,242.000000,"[-2703, -2690, -2701, -2726, -2720, -2701, -27...",True,True,True,0709_g0
2,61775,70325,1235500,1406500,"[34.87362769543909, 35.6706262340847, 18.02458...",21.222267,271.346975,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,8550,"[[61775, 61810], [62245, 62280], [63085, 63130...",17,"[1, 10, 80, 96, 180, 188, 203, 262, 270, 438, ...",47,181.914894,"[-210, -209, -209, -186, -204, -176, -188, -17...",True,True,True,0709_g0
3,81850,93050,1637000,1861000,"[44.421253705652624, 85.13879158820612, 34.138...",21.222267,179.841336,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,11200,"[[81850, 81880], [83065, 83100], [83420, 83455...",13,"[1, 16, 77, 89, 152, 164, 175, 232, 244, 255, ...",69,162.318841,"[-1076, -1089, -1089, -1087, -1074, -1085, -10...",True,True,True,0709_g0
4,96090,101120,1921800,2022400,"[10.181296921242865, 93.68222195050916, 60.404...",21.222267,180.797063,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,5030,"[[96090, 96145], [96835, 96865], [96895, 96930...",8,"[4, 74, 82, 89, 104, 150, 162, 232, 245, 253, ...",28,179.642857,"[-467, -468, -477, -492, -499, -492, -444, -44...",True,True,True,0709_g0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,18218469,18223479,364369380,364469580,"[3.8887064750841964, 4.543270917366302, 8.5394...",12.549355,303.313564,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,5010,"[[18218469, 18218514], [18218734, 18218764], [...",7,"[5, 56, 97, 162, 193, 202, 281, 323, 346, 368,...",26,192.692308,"[-47, -66, -37, -53, -61, -66, -39, -40, -41, ...",True,True,True,713_g0
188,18234059,18242729,364681180,364854580,"[3.2825189335835696, 41.6172687192349, 43.4294...",12.549355,307.420424,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,8670,"[[18234059, 18234094], [18234309, 18234354], [...",12,"[5, 51, 83, 105, 161, 208, 268, 302, 311, 322,...",43,201.627907,"[-62, -36, -21, -31, 16, 19, -47, -52, -58, -3...",True,True,True,713_g0
189,18252874,18273224,365057480,365464480,"[5.661038630094994, 71.5871505671681, 98.33510...",12.549355,345.299606,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,20350,"[[18252874, 18252914], [18253299, 18253334], [...",35,"[2, 11, 21, 79, 86, 105, 140, 152, 160, 202, 2...",128,158.984375,"[-3947, -3977, -3970, -3951, -4023, -3975, -39...",True,True,True,713_g0
190,18303094,18313359,366061880,366267180,"[21.683543331608792, 41.82099792411847, 28.592...",12.549355,237.611099,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,10265,"[[18303094, 18303129], [18303664, 18303724], [...",19,"[6, 17, 33, 49, 62, 103, 114, 125, 148, 158, 2...",60,171.083333,"[684, 702, 675, 706, 698, 701, 718, 700, 690, ...",True,True,True,713_g0


#### compute the spectrograms

In [18]:
# but don't compute if they were already computed
# try:
#     bout_pd = pd.read_pickle(sess_bouts_file) ## make it jump if there is no sess_bouts file
#     one_spec = bout_pd['spectrogram'][0] ## make it jump if there is no spectrogram key
#     logger.info('loaded bout pandas with spectrograms from ' + sess_bouts_file)
# except KeyError:
#     logger.info('Computing spectrograms')
#     bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
#     logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
#     bout_pd.to_pickle(sess_bouts_file)

In [19]:
# just compute them alright?
logger.info('Computing spectrograms')
bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
bout_pd.to_pickle(sess_bouts_file)

2021-12-03 17:35:50,604 root         INFO     Computing spectrograms
2021-12-03 17:36:26,317 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/bouts_sglx/bout_sess_auto.pickle


In [20]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,21460,32220,429200,644400,"[8.713733042828666, 130.93567831255595, 233.87...",21.222267,359.953578,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,10760,...,17,"[2, 30, 105, 128, 172, 180, 198, 211, 540, 609...",58,185.517241,"[1065, 1076, 1081, 1095, 1083, 1103, 1086, 108...",True,True,True,0709_g0,"[[0.09833268862151824, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,40125,54645,802500,1092900,"[13.096523956130605, 9.062461794160846, 31.203...",21.222267,310.517498,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,14520,...,18,"[4, 11, 18, 178, 187, 200, 400, 414, 432, 465,...",60,242.000000,"[-2703, -2690, -2701, -2726, -2720, -2701, -27...",True,True,True,0709_g0,"[[0.24300522397366095, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [21]:
bout_pd['file'][0]

'/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/0709_g0/wav_mic.wav'

## inspect the bouts and curate them

#### visualize one bout

In [22]:
bout_pd.iloc[0]

start_ms                                                      21460
end_ms                                                        32220
start_sample                                                 429200
end_sample                                                   644400
p_step            [8.713733042828666, 130.93567831255595, 233.87...
rms_p                                                     21.222267
peak_p                                                   359.953578
bout_check                                                    False
file              /mnt/sphere/speech_bci/derived_data/s_b1253_21...
len_ms                                                        10760
syl_in            [[21460, 21495], [22305, 22340], [24145, 24190...
n_syl                                                            17
peaks_p           [2, 30, 105, 128, 172, 180, 198, 211, 540, 609...
n_peaks                                                          58
l_p_ratio                                       

In [23]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [24]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bout_pd.iloc[24])

    'data': [{'type': 'scatter',
              'uid': '5a22f824-8088-48e8-a451-c…

In [25]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,21460,32220,429200,644400,"[8.713733042828666, 130.93567831255595, 233.87...",21.222267,359.953578,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,10760,...,17,"[2, 30, 105, 128, 172, 180, 198, 211, 540, 609...",58,185.517241,"[1065, 1076, 1081, 1095, 1083, 1103, 1086, 108...",True,True,True,0709_g0,"[[0.09833268862151824, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,40125,54645,802500,1092900,"[13.096523956130605, 9.062461794160846, 31.203...",21.222267,310.517498,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,14520,...,18,"[4, 11, 18, 178, 187, 200, 400, 414, 432, 465,...",60,242.000000,"[-2703, -2690, -2701, -2726, -2720, -2701, -27...",True,True,True,0709_g0,"[[0.24300522397366095, 0.0, 0.0, 0.0, 0.0, 0.0..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [26]:
bout_pd.reset_index(drop=True, inplace=True)

In [27]:
## Set confusing by default, will only be False once asserted bout/or not
bout_pd['confusing'] = True
bout_pd['bout_check'] = False

In [28]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [29]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bout_pd)

### save it

In [30]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': 'read_wav_chan',
 'file_order_fun': 'sess_file_id',
 'min_segment': 30,
 'min_silence': 2000,
 'min_bout': 5000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 120000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle',
 'sample_rate': 20000}

In [31]:
### get the curated file path
##save to the curated file path
viz_bout.bouts_pd.to_pickle(sess_bouts_curated_file)
logger.info('saved curated bout pandas to pickle {}'.format(sess_bouts_curated_file))

2021-12-03 17:44:59,743 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/bouts_sglx/bout_curated.pickle


### summary of where the good bouts are along the session

In [32]:
def give_summary(bpd: pd.DataFrame):
    all_files = np.unique(bpd['file'])
    summary_dict = {f: np.where((bpd['bout_check']==True) & (bpd['file']==f))[0].size for f in all_files}
    return summary_dict

sum_dict = give_summary(viz_bout.bouts_pd)
sum_dict

{'/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/0709_g0/wav_mic.wav': 0,
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/1227_g0/wav_mic.wav': 1,
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/1819_g0/wav_mic.wav': 1,
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/sglx/713_g0/wav_mic.wav': 3}

## Some helper functions for somewhere down the line

In [54]:
sess_par_e = dict.copy(sess_par)
sess_par_e['epoch'] = '0610_g0' 

In [61]:
bpd.head()

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,2764745,2770765,69118625,69269125,"[137.18250603801323, 166.1364744613515, 46.022...",12.20809,338.931800,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6020,...,14,"[1, 25, 35, 48, 57, 75, 275, 285, 349, 358, 37...",49,122.857143,"[-74, -123, -103, -104, -91, -103, -85, -79, -...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,2821180,2830650,70529500,70766250,"[73.89770606840956, 76.71229477152468, 50.9882...",12.20809,323.989772,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,9470,...,10,"[1, 49, 87, 123, 131, 141, 168, 197, 210, 218,...",47,201.489362,"[33, 36, 31, 17, 38, 26, 35, 24, 21, 34, 27, 2...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2890955,2897000,72273875,72425000,"[5.723650048239091, 20.093294098403362, 40.008...",12.20809,385.427078,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6045,...,18,"[12, 19, 26, 37, 49, 155, 169, 181, 196, 208, ...",58,104.224138,"[37, 34, 48, 23, 45, 44, 48, 56, 47, 64, 55, 5...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3022775,3030175,75569375,75754375,"[10.760223359304039, 14.367538094139961, 38.22...",12.20809,250.017320,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,7400,...,25,"[2, 28, 37, 86, 100, 111, 137, 144, 157, 173, ...",64,115.625000,"[-105, -115, -130, -129, -143, -135, -149, -14...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,3052560,3060680,76314000,76517000,"[84.90921288692883, 71.8622817106007, 22.56276...",12.20809,349.395052,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,8120,...,23,"[4, 14, 73, 89, 100, 116, 136, 148, 160, 171, ...",74,109.729730,"[-27, -40, -36, -37, -40, -32, -32, -28, -30, ...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [92]:
reload(et)
bpd = et.get_epoch_bout_pd(sess_par_e, only_curated=True)
bpd.head()

2021-09-24 18:39:39,171 ceciestunepipe.file.bcistructure INFO     loading curated bouts for session 2021-07-18 from /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/bouts_ceciestunepipe/bout_curated.pickle
2021-09-24 18:39:42,038 ceciestunepipe.file.bcistructure INFO     Filtering bouts for epoch 0610_g0
2021-09-24 18:39:42,196 ceciestunepipe.file.bcistructure INFO     Filtering also only manually curated bouts


In [94]:
bpd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram,bird,sess
0,6300478,6351243,157511950,158781075,"[7.886206128106227, 15.074117306094267, 18.369...",22.351748,553.969334,True,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,50765,...,829,61.236429,"[13, -7, 0, 9, -8, 2, -7, -10, -4, -13, -1, 2,...",False,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",s_b1253_21,2021-07-18


In [96]:
bpd['start_ms'].values

array([ 6300478,  7102501,  7180616,  7245376,  7661606,  8031581,
        8128521,  8202321,  8442876,  9853004, 11014954, 12578659,
       14212972, 15410472, 15496242, 17104694, 18395779, 20073717,
       20107052, 21557542, 21939152, 23513780, 23676120, 23753090,
       23766770, 24457160, 24816190, 27239488, 27308728, 27405768,
       27495628, 27737218, 28437448, 29355223])

In [97]:
bpd['end_ms'].values - bpd['start_ms'].values

array([50765, 38435, 31900, 44620, 55725, 47565, 39805, 34070, 60000,
       45875, 45460, 41365, 39195, 58425, 52140, 46640, 21695, 17460,
       39010, 47655, 52790, 23410, 42345,  9165, 18225, 51855, 57915,
       34420, 42935, 35810, 40380, 51640, 63155, 19065])

In [1]:
bpd

NameError: name 'bpd' is not defined